# Application and Data Portability Setup Notebook

This notebook provisions SQL ADP services to an Azure Subscription for migration purposes such as importing and exporting Azure SQL Server database(s). The following scenarios are covered:

-   [x]  Azure to Azure migration
-   [x]  Azure to Air-gapped Azure (highly secured Azure offer)
-   [x]  Azure to !Azure (3rd party or on-premises)

This bootstrapper is designed to be ran from a client workstation or laptop with minimal modification. The notebook will attempt to locate three different Azure Resources in a customer subscription. If not found it will install the following to a given Resource Group. If the resources group specified doesn't exist, it will create a new one:

1.  A **Custom Azure ADP Function** to orchestrate the Azure Batch and Storage (see \[Github\](........\\Program Files\\Azure Data Studio\\resources\\app\\out\\vs\\code\\electron-browser\\workbench\\workbench.html))
2.  An Azure **Storage Account** for storage
3.  An **Azure Batch** resource

This notebook will help accomplish the below steps as a part of Bootstrapping an ADP Resource Group

![](VisualBootstrapperNB.PNG)

1.  Set Environment Variables
2.  Define Notebook Functions
3.  MS Azure Login
4.  Validate/ Create ADP Resource Group
5.  Provision Virtual Network
6.  Provision ADP Storage Account
7.  Provision ADP Portability Function
8.  Provision ADP Batch Account
9.  Provision Contributor access (RBAC) to Portability function over ADP Resource Group

Setup _will not_ perform tasks on existing Azure assets such as Microsoft SQL Servers, databases, VMs, etc. Other notebooks cover the <u><a href="import-sql-server.ipynb" rel="nofollow">import</a></u> or <u><a href="export-sql-server.ipynb" rel="nofollow">export</a></u> operations. This notebook will only check for the existence of these ADP resources and their setup.

In [ ]:
# Setup client environment variables that the rest of the notebook will use
$AdpResourceGroup    = ""                                           # Target Resource Group to bootstrap with ADP components - A new one will be created if the specified Resource Group doesn't exist
$AdpRegion        = "eastus"                                                # Region/Location of the resource group to be bootstrapped

# Derived settings
$AdpSubscription     = ""                                                      # Target Azure Subscription Name or ID to bootstrap data portability resources
$AdpFunc             = $AdpResourceGroup + "Control"
$AdpStorage          = $AdpResourceGroup.ToLower() + "storage"
$AdpBatch            = $AdpResourceGroup.ToLower() + "batch"
$AdpVNET             = $AdpResourceGroup + "VNet"

# Bootstrapper URLs - Update with the recommended toolkit version and build
$BaseToolkitUrl         =   "https://hybridtoolkit.blob.core.windows.net/components"
$ReleaseVersion         =   "0.13"
$BuildNumber            =   "74938"
$AdpDownloadUrl =   "$BaseToolkitUrl/$ReleaseVersion/ADPControl-$BuildNumber.zip"
$AdpWrapperUrl =   "$BaseToolkitUrl/$ReleaseVersion/BatchWrapper-$BuildNumber.zip"

Write-Output "Setting the Environment:"
Get-ChildItem Env: | Where-Object Name -Match "BOOTSTRAP"

## Define Helper Functions
Defines logical functions for the rest of the notebook. Function blocks are combined in a single cell that can be collapsed for readability or expanded for further examination. Nothing is executed until called later in the notebook. As a result, this cell is a requirement for any of the other cells below it. 

In [ ]:
# Expand cell to view framework

function Get-ConnectionStringProperties
{
    param([Parameter (Mandatory=$true)] [string]$ConnectionString)
    $hash = @{}
    $attributes = $ConnectionString.Split(";");  
    foreach($at in $attributes)
    {
        $attributeKeyValue=$at.Split("=",2)    
        $hash[$attributeKeyValue[0]]=$attributeKeyValue[1]    
    }$hash    
}

function Login-Azure
{    
    # query azure locations to test for existing az login session exists with valid access tocken
    $azureLocations = az account list-locations -o JSON 2>$null | ConvertFrom-Json
    
    if (!$azureLocations){ #If there are no az locations, there is no existing az login session
        $subscriptions = az login -o JSON | ConvertFrom-Json      # Login   
    }
    else {
        $subscriptions = az account list -o JSON | ConvertFrom-Json # getting subscriptions for the user to use in gridview
    }

    if(![string]::IsNullOrWhiteSpace($AdpSubscription)) #If there is a subscription specified by user in the variables section
    {
        $specified_Subscription= az account show --subscription $AdpSubscription -o json |ConvertFrom-Json 
        if (!$specified_Subscription) #if specified subscription is not valid
        {       
            $currentUser= az ad signed-in-user show --query "{displayName:displayName,UPN:userPrincipalName}" -o json|ConvertFrom-Json # get current logged in user infomration
            Write-Host "Refer below for the list of subscriptions for logged in account '$($currentUser.UPN)'`n"
            az account list --query "[].{Name:name,SubscriptionID:id}" -o table          # list subscriptions under current logged in account
        }
        else { # if specified subscription is valid
            Write-Output "Using subscription... '$($specified_Subscription.name)' ... '$($specified_Subscription.id)'" 
        }
    }
    else { # if no subscription is specified, users are given a gridview to select subscription from

        $selectedSubscription = $subscriptions | Select-Object -Property Name, Id | Out-GridView -PassThru
        $SubscriptionId = $selectedSubscription.Id
        $Subscription = $selectedSubscription.Name   
        $AdpSubscription = $subscription  
        Write-Output "Using subscription... '$AdpSubscription' ... '$SubscriptionId'"    
    }    
}

function Bootstrap-AzResourceGroup
{
    [CmdletBinding()]
    param(
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$Subscription,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$ResourceGroupName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$ResourceGroupLocation    
    )   
     
    #Validating parameter $Subscription
    $specified_Subscription = az account show --subscription $Subscription -o json |ConvertFrom-Json
    if(!$specified_Subscription){
        $currentUser= az ad signed-in-user show --query "{displayName:displayName,UPN:userPrincipalName}" -o json|ConvertFrom-Json # get current logged in user infomration
        Write-Host "Refer below for the list of subscriptions for logged in account '$($currentUser.UPN)'`n"
        az account list --query "[].{Name:name,SubscriptionID:id}" -o table          # list subscriptions under current logged in account
        return
        }
    #Validating parameter $ResourceGroupLocation
    $specified_RGLocation = az account list-locations --query "[?name == '$ResourceGroupLocation']" -o JSON | ConvertFrom-Json
    if(!$specified_RGLocation) {
        Write-Host "ResourceGroupLocation '$ResourceGroupLocation' not found. Refer below for the list of locations for the account:`n" 
        az account list-locations --query "[].{Locations: name}" -o table
        return
    }
    # check if Resource group already exists under specified subscription
    $existingResourceGroup = az group show -n $ResourceGroupName --subscription $Subscription -o JSON 2>$null  | ConvertFrom-Json

    if ($existingResourceGroup){ #Found specified resource group        
        Write-Output "Found Resource Group '$($existingResourceGroup.name)'. "   
        } 
    else { 
        # Resource group not found under specified subscription. Create new Resource group 
        Write-Output "Resource group '$($ResourceGroupName)' not found. `nCreating new resource group ... "
        $newResourceGroup = az group create -n $ResourceGroupName -l $ResourceGroupLocation --subscription $Subscription -o JSON | ConvertFrom-Json
        Write-Output "Created Resource Group '$($newResourceGroup.name)'"
        }  
}

function Bootstrap-AzVirtualNetwork
{    
    [CmdletBinding()]
    param(
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$Subscription,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$ResourceGroupName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$VNetName,  
        [Parameter (Mandatory=$false)] [ValidateNotNullOrEmpty()] [string]$SubNetName = "default"         
    )
    
    #Validating parameter $ResourceGroupName
    $specified_RG = az group show -n $ResourceGroupName --subscription $Subscription 2>$null
    if(!$specified_RG) {
        Write-Host "Invalid Input: ResourceGroupName. Resource Group $ResourceGroupName not found"
        return
        }   
    
     #Check if default subnet exists under specified Vnet 
     $existingVnetSubnet =  az network vnet subnet show -n $SubNetName --vnet-name $VNetName -g $ResourceGroupName --subscription $Subscription -o JSON |ConvertFrom-Json
     if ($existingVnetSubnet){ #Found default subnet under specified VNet
         Write-Output "Found default Subnet under specified VNet. `nSubNet ID '$($existingVnetSubnet.id)'"
     } 
     else { 
         #VNet or defaut subnet not found under specified resource group. Create new VNet with default Subnet /Add default subnet to existing VNet
         Write-Output "Creating new Virtual network with default Subnet ID ... "
         $newVNet = az network vnet create --name "$AdpVNET" --resource-group $AdpResourceGroup --subscription $AdpSubscription  --subnet-name $SubNetName  -o JSON  |ConvertFrom-Json #vnet create/Update command: Bug: In this command, the output variable is not getting converted to PS objects.
         $newVNet =  az network vnet subnet show -g $AdpResourceGroup --vnet-name $AdpVNET -n $SubNetName --subscription $AdpSubscription -o JSON |ConvertFrom-Json # added this line due to above bug
         Write-Output "Created VNet with default Subnet - ID: '$($newVNet.id)'"
     }
}

function Bootstrap-AzStorageAccount
{    
    [CmdletBinding()]
    param(
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$Subscription,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$ResourceGroupName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$StorageAccountName,
        [Parameter (Mandatory=$false)] [ValidateNotNullOrEmpty()] [string]$StorageAccountKind = "StorageV2",
        [Parameter (Mandatory=$false)] [ValidateNotNullOrEmpty()] [string]$StorageAccountSKU = "Standard_LRS"           
    )

    $ListStorageAccountKind = "BlobStorage", "BlockBlobStorage", "FileStorage", "Storage", "StorageV2"
    $ListStorageAccountSKU = "Premium_LRS", "Premium_ZRS", "Standard_GRS", "Standard_GZRS", "Standard_LRS", "Standard_RAGRS", "Standard_RAGZRS", "Standard_ZRS"

    
     #Validating parameter $ResourceGroupName
     $specified_RG = az group show -n $ResourceGroupName --subscription $Subscription 2>$null
     if(!$specified_RG) {
         Write-Host "Invalid Input: ResourceGroupName. Resource Group $ResourceGroupName not found"
         return
         } 
    #Validating parameter $StorageAccountKind
    if($StorageAccountKind -notin $ListStorageAccountKind) {
        Write-Host "StorageAccountKind '$StorageAccountKind' not valid. Accepted values: $ListStorageAccountKind"  
        return
        }
    #Validating parameter $StorageAccountSKU
    if($StorageAccountSKU -notin $ListStorageAccountSKU) {
        Write-Host "StorageAccountSKU '$StorageAccountSKU' not valid. Accepted values: $ListStorageAccountSKU"  
        return
        }
    
    #Check if storage account exists under specified resource group    
    $existingStorageAccount = az storage account show -n $StorageAccountName -g $ResourceGroupName --subscription $Subscription  -o JSON| ConvertFrom-Json

    if ($existingStorageAccount){ #Found specified storage account
        Write-Output "Found storage account '$($existingStorageAccount.name)'"
    } 
    else { 
        #Storage account not found under specified resource group. Create new storage account
        Write-Output "Creating new storage account ... "
        $newStorageAccount = az storage account create -n $StorageAccountName --kind $StorageAccountKind --sku $StorageAccountSKU -g $ResourceGroupName --subscription $Subscription -o JSON | ConvertFrom-Json
        Write-Output "Created Storage Account '$($newStorageAccount.name)'"
    }
}

function Bootstrap-AzFunctionApp
{
    [CmdletBinding()]
    param(
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$Subscription,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$ResourceGroupName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$FunctionName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$StorageAccountName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$FunctionAppPackageURL,
        [Parameter (Mandatory=$false)] [ValidateNotNullOrEmpty()] [string]$FunctionAppVersion="3",
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$ConsumptionPlanLocation         
    )   
     #Validating parameter $ResourceGroupName
     $specified_RG = az group show -n $ResourceGroupName --subscription $Subscription 2>$null
     if(!$specified_RG) {
         Write-Host "Invalid Input: ResourceGroupName. Resource Group $ResourceGroupName not found"
         return
         } 
    #Validating parameter StorageAccountName
    $specified_storageAccount = az storage account show -n $StorageAccountName -g $ResourceGroupName --subscription $Subscription 2>$null
    if(!$specified_storageAccount){ 
        Write-Host "Invalid Input: StorageAccount name. Storage Account $StorageAccountName not found"                  
        return
    }
    #Validating parameter FunctionAppVersion
    if($FunctionAppVersion -notin (2,3)){
        Write-Host "FunctionAppVersion '$FunctionAppVersion' not valid. Accepted values: 2,3"  
        return
    }
    #Validating parameter ConsumptionPlanLocation
    $specified_Location = az functionapp list-consumption-locations --query "[?name == '$ConsumptionPlanLocation']" -o JSON | ConvertFrom-Json 
    if(!$specified_Location) { 
        Write-Host "ComsumptionPlanLocation '$ConsumptionPlanLocation' not valid. Refer below for available options: `n" 
        az functionapp list-consumption-locations --query "[].{ConsumptionPlanLocations: name}" -o table
        return
    }
    
    #Check if functionapp exists within the specified resource group
    $existingFunctionApp = az functionapp show -n $FunctionName -g $ResourceGroupName --subscription $Subscription -o JSON 2>$null | ConvertFrom-Json

    if ($existingFunctionApp){  #ensure that function application is linked to the specified storage account

        #retrieve connection string of storage account linked to FunctionApp and check if the connected storage account is equal to the storage account passed to here
        $funcApp_Storage_ConnString= az functionapp config appsettings list -n $FunctionName -g $ResourceGroupName --subscription $Subscription --query "[?name == 'AzureWebJobsStorage'].value" -o JSON 2>$null | ConvertFrom-Json
        $connectionString = Get-ConnectionStringProperties -ConnectionString "$funcApp_Storage_ConnString"
        $isConnected = $connectionString.AccountName -eq $StorageAccountName

        if($isConnected) { 
            Write-Output "Found FunctionApp '$FunctionName'. `nConnected storage account is '$StorageAccountName'"
        } 
        else {
            Write-Output "Found FunctionApp '$FunctionName'. `nConnected storage account is NOT '$StorageAccountName', but '$($connectionString.AccountName)' "
        }
    } 
    else { #Provision FunctionApp 

        $newFuncApp= az functionapp create  --name $FunctionName --resource-group $ResourceGroupName --subscription $Subscription --storage-account $StorageAccountName --functions-version $FunctionAppVersion --consumption-plan-location $ConsumptionPlanLocation -o JSON | ConvertFrom-JSON 
        Write-Output "`nCreated Function Application '$($newFuncApp.name)'"
        $AppInsightsInstrumentationKey =  az functionapp config appsettings list --name $FunctionName --resource-group $ResourceGroupName --subscription $Subscription --query "[?name == 'APPINSIGHTS_INSTRUMENTATIONKEY'].value" -o JSON | ConvertFrom-JSON        
        az functionapp config appsettings set --name $FunctionName --settings "WEBSITE_RUN_FROM_PACKAGE=$FunctionAppPackageURL" "APPLICATIONINSIGHTS_CONNECTION_STRING=InstrumentationKey=$AppInsightsInstrumentationKey" `
                                               --resource-group $ResourceGroupName --subscription $Subscription -o none 
        Write-Output "Set '$($newFuncApp.name)' to run from package '$FunctionAppPackageURL'"
        az functionapp identity assign -n $FunctionName -g $ResourceGroupName --subscription $Subscription -o none  
        Write-Output "Enabled system assigned managed identity for  '$($newFuncApp.name)'"        
    }
}

function Bootstrap-AzBatchAccount
{
    [CmdletBinding()]
    param(
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$Subscription,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$ResourceGroupName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$BatchAccountLocation,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$BatchAccountName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$StorageAccountName,
        [Parameter (Mandatory=$true)] [ValidateNotNullOrEmpty()] [string]$ApplicationPackageURL,
        [Parameter (Mandatory=$false)] [ValidateNotNullOrEmpty()] [string]$ApplicationName="SqlPackageWrapper",  
        [Parameter (Mandatory=$false)] [ValidateNotNullOrEmpty()] [string]$ApplicationPackageVersionName="1"   
    )
    #Validating parameter $ResourceGroupName
    $specified_RG = az group show -n $ResourceGroupName --subscription $Subscription 2>$null
    if(!$specified_RG) {
        Write-Host "Invalid Input: ResourceGroupName. Resource Group $ResourceGroupName not found"
        return
        } 
   #Validating parameter StorageAccountName
   $specified_storageAccount = az storage account show -n $StorageAccountName -g $ResourceGroupName --subscription $Subscription 2>$null
   if(!$specified_storageAccount){ 
       Write-Host "Invalid Input: StorageAccount name. Storage Account $StorageAccountName not found"                  
       return
   }
    #Validating parameter $BatchAccountLocation
    $specified_BLocation = az account list-locations --query "[?name == '$BatchAccountLocation']" -o JSON | ConvertFrom-Json
    if(!$specified_BLocation) {
        Write-Host "BatchAccountLocation '$BatchAccountLocation' not found. Refer below for the list of locations for the account:`n" 
        az account list-locations --query "[].{Locations: name}" -o table
        return
        }    

    $SQLPackageWrapperLocal = "$Env:USERPROFILE" + "\Downloads\batchApplicationPackage.zip"   #path to download application package
    $downloaded= Test-Path $SQLPackageWrapperLocal 

    #check if Batch account exists in the specified resource group
    $existingBatchAccount = az batch account show -n $BatchAccountName -g $ResourceGroupName --subscription $Subscription -o JSON 2>$null | ConvertFrom-Json

    if ($existingBatchAccount){  #Batch Account exists, 
        Write-output "Found Batch Account '$BatchAccountName'"

        #checking if storage account is set for Batch Account
        #$existingStorageAccount = az storage account show -n $StorageAccountName -g $ResourceGroupName --subscription $Subscription -o JSON 2>$null | ConvertFrom-Json        # check if storage account set is ADP Storage A/c
        $connectedToStorage= $existingBatchAccount.autoStorage        #.storageAccountId -eq $existingStorageAccount.id 

        # check if latest application package is installed       
        $appPackageInstalled = az batch application package show --application-name $ApplicationName --version-name $ApplicationPackageVersionName -n $BatchAccountName -g $ResourceGroupName --subscription $Subscription -o JSON 2>$null | ConvertFrom-Json

        if($connectedToStorage -and $appPackageInstalled){ # BatchAccount connected to specified storageaccount and latest applicationpackage is installed
            $connectedStorageAccName = $existingBatchAccount.autoStorage.storageAccountId.Split("/")[-1]
            Write-Output "Already connected to Storage account '$connectedStorageAccName' `nFound latest application package installed. `nReusing existing Resources"
        }
        else {
            if(!$connectedToStorage){ # Connect to ADPStorageAccount               

                az batch account set --storage-account $StorageAccountName -n $BatchAccountName --resource-group $ResourceGroupName --subscription $Subscription -o none | ConvertFrom-JSON 
                Write-Output "No Storage Account Found. Connected to storage account '$StorageAccountName'"
            }
            if(!$appPackageInstalled) { #application package is not installed, installing application package               
                if(!$downloaded) { Invoke-WebRequest -Uri $ApplicationPackageURL -OutFile $SQLPackageWrapperLocal } #downloads package to current user's downloads folder
                az batch application package create --application-name $ApplicationName --version-name $ApplicationPackageVersionName --package-file $SQLPackageWrapperLocal --name $BatchAccountName --resource-group $ResourceGroupName --subscription $Subscription -o none
                Write-Output "No Application package found. Installed application package under '$ApplicationName'"
            }
            Write-Output "Batch Account '$BatchAccountName' is provisioned as required"
        }
    } else { #Provision Batch Account

        $newBatchAccount = az batch account create -n $BatchAccountName -l $BatchAccountLocation --resource-group $ResourceGroupName --subscription $Subscription -o JSON | ConvertFrom-JSON
        Write-Output "Created Batch Account '$($newBatchAccount.name)'" 
        az batch account set --storage-account $StorageAccountName -n $BatchAccountName --resource-group $ResourceGroupName --subscription $Subscription -o none   
        Write-Output "Set Storage Account to '$StorageAccountName'"       
        if(!$downloaded) { Invoke-WebRequest -Uri $ApplicationPackageURL -OutFile $SQLPackageWrapperLocal } #downloads package to current user's downloads folder
        az batch application package create --application-name $ApplicationName --version-name $ApplicationPackageVersionName --package-file $SQLPackageWrapperLocal --name $BatchAccountName --resource-group $ResourceGroupName --subscription $Subscription -o none  
        Write-Output "Installed application package '$ApplicationName'"              
        
    }    
}

function Bootstrap-FuncRBAC {
    [CmdletBinding()]
    param (
        [Parameter(Mandatory=$true)][ValidateNotNullOrEmpty()][string]$Subscription,
        [Parameter(Mandatory=$true)][ValidateNotNullOrEmpty()][string]$ResourceGroupName,
        [Parameter(Mandatory=$true)][ValidateNotNullOrEmpty()][string]$AzFunctionName,
        [Parameter(Mandatory=$false)][ValidateNotNullOrEmpty()][string]$Role="Contributor"
    )

    # Get the resource group's scope
    $rg = az group show --resource-group $ResourceGroupName --subscription $Subscription | ConvertFrom-Json
    if (!$rg) {
        Write-Error "Failed to setup RBAC for $AzFunctionName... Resource Group '$ResourceGroupName' not found on subscription '$Subscription'"
        exit
    }

    $scope = $rg.id
    Write-Output "Found scope... $scope"
    
    # Get the az function principal id
    $app = az functionapp show -n $AzFunctionName --resource-group $ResourceGroupName --subscription $Subscription | ConvertFrom-Json
    if (!$app) {
        Write-Error "Failed to setup RBAC for $AzFunctionName... Az function not found!"
        exit
    }

    $prid = $app.identity.principalId
    Write-Output "Found principal id of Azure function... $prid"

    # Verify if a role assignment has been created
    $assignments = az role assignment list --subscription $Subscription -g $ResourceGroupName | ConvertFrom-Json

    # Get the correct assignment
    $svcPrincipal = $assignments | Where-Object {($_.scope -eq $scope) -and ($_.principalType -eq "ServicePrincipal") -and ($_.principalId -eq $prid)}
    $id = $svcPrincipal.id
    $def = $svcPrincipal.roleDefinitionName

    Write-Output "Found service principal role... '$def'"    
    # Check if the right role is assigned
    if ($def -eq $Role)
    {            
        Write-Output "Service Principle '$prid' found with '$Role' role. No work needed"
        return
    }

    # Continue to setup RBAC, once we verify an assignment is not setup and all the resources exist
    Write-Output "Creating new role assignment by running: 'az functionapp identity assign -g $ResourceGroupName -n $AzFunctionName --scope $scope --subscription $Subscription'"
    Write-Warning "If your account does not have the access to assign new roles as Owner or User Access Administrator for the resource group, than you will need to contact your Azure AD Administrator to assign a service principle using the commands above"    
    az functionapp identity assign -g $ResourceGroupName -n $AzFunctionName --scope $scope --subscription $Subscription 
}

Write-Host "Helper Functions Created successfully"  

## Connect to Azure and Select Subscription
Run the below cell to login to an Azure account. <b>Be sure to check the Windows Taskbar for a subscription selection dialog box.</b>

_Note: the dialog box window may appear behind the active Azure Data Studio window._

In [ ]:
Login-Azure

## Create Resource Group
The Data Portability Notebooks provision several resources to enable import and export of Azure SQL DB databases and Azure SQL Managed Instance databases.  These resources include Storage, Azure Batch, Azure Function resources and their dependent Azure resources.  All resources created by this notebook are associated with a common Azure Resource Group to enable easy management of resources related to Data Portability.

_Note: this notebook will reuse the Resource Group specified if it already exists._

In [ ]:
Bootstrap-AzResourceGroup -ResourceGroupName $AdpResourceGroup -ResourceGroupLocation $AdpRegion -Subscription $AdpSubscription

## Provision Virtual Network

The Data Portability resources include Virtual Machines created to execute the database import and export operations.  To provision this in a secure mode, this notebook will provision a Virtual Network to attach the Virtual Machines to.  These VMs do not need to be directly accessed so no publicly addressable ports should be created.

In [ ]:
Bootstrap-AzVirtualNetwork -VNetName $AdpVNET -ResourceGroupName $AdpResourceGroup -Subscription $AdpSubscription

## Provision Storage Account

The Data Portability resources require a Storage Account to attach the Virtual Machines to.  This storage account is used for maintaining log files and other Azure Batch related information.  This Storage Account can be the same or different than the Storage Acount used during Import and Export operations to store the logical database (.bacpac) files.

In [ ]:
Bootstrap-AzStorageAccount -StorageAccountName $AdpStorage -ResourceGroupName $AdpResourceGroup -Subscription $AdpSubscription

## Provision Data Portability Orchestrator

The Data Portability resources include an orchestration component which coordinates the import and export operations.  The orchestrator is built as an Azure Function and provides the REST APIs that the Import and Export notebooks use to either import or export an Azure SQL DB Server and its databases.

In [ ]:
Bootstrap-AzFunctionApp -FunctionName $AdpFunc -StorageAccountName $AdpStorage -FunctionAppPackageURL $AdpDownloadUrl `
    -ConsumptionPlanLocation $AdpRegion -ResourceGroupName $AdpResourceGroup -Subscription $AdpSubscription

## Provision Hosted SqlPackage Application
To Data Portability solution leverages SqlPackage and DacFx to perform the import and export of each database.  To host SqlPackage the solution uses an Azure Batch application.  Azure Batch provides the ability to create a pool of Virtual Machines that can perform the import and export operations.  This notebook will provision a pool of 2 virutal machines.  The machines are not actually allocated until the first Import or Export operation is performed using the Import or Export notebooks.  

In [ ]:
Bootstrap-AzBatchAccount -BatchAccountName $AdpBatch -StorageAccountName $AdpStorage -BatchAccountLocation $AdpRegion `
    -ApplicationPackageURL $AdpWrapperUrl -ResourceGroupName $AdpResourceGroup  -Subscription $AdpSubscription

## Grant Data Portability Controller Access to Data Portability Resources
The Data Portability orchestrator requires Contributor access to the other Data Portability resources created.  To do this, Azure RBAC (role based access control) is used to grant access to the orchestrator so that it can communicate with the hosted SqlPackage application.  The function below will verify a Service Principal was created with the proper scope or attempt to create the role assignment. Any user executing this notebook will need to have Owner or User Access Administrator permissions to the Resource Group to assign the permission. Otherwise, contact your Azure AD Administrator. 

In [ ]:
Bootstrap-FuncRBAC -AzFunctionName $AdpFunc -ResourceGroupName $AdpResourceGroup -Subscription $AdpSubscription